Optional, Any And Defaults

In [8]:
from pydantic import BaseModel, ValidationError

class User(BaseModel):
    name: str
    age: int
    
try:
    user1 = User(name="Alice") # exception occurs because age is required
    print(user1.age)
except ValidationError as e:
    print(e)

1 validation error for User
age
  Field required [type=missing, input_value={'name': 'Alice'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.5/v/missing


In [9]:
from pydantic import BaseModel, ValidationError
from typing import Optional

class User(BaseModel):
    name: str
    age: Optional[int] = None
    
try:
    user1 = User(name="Alice")
    print(user1.age)
except ValidationError as e:
    print(e)

None


In [10]:
from typing import Any

class Mnemonic(BaseModel):
    requiredInt: int
    optionalInt: int = 10
    optionalNullableInt: Optional[int] = 10
    optionalAnyType: Any = 10

In [11]:
m = Mnemonic(requiredInt=22, optionalNullableInt=None,
             optionalAnyType="This value could be literally anything including None")
m

Mnemonic(requiredInt=22, optionalInt=10, optionalNullableInt=None, optionalAnyType='This value could be literally anything including None')

UUIDs And Default Factories

In [18]:
import uuid

class User(BaseModel):
    id: uuid.UUID
    name: str

try:
    user = User(id=uuid.uuid4(), name="Allison")
    # user = User(id="asdf-qwer-1234-zxcv", name="Allison") # would return an exception because valid UUID is expected here
    print(user)
except ValidationError as e:
    print(e)

id=UUID('29aef591-7708-42de-97c2-884fc88fe5de') name='Allison'


In [19]:
from pydantic import Field
class User(BaseModel):
    id: uuid.UUID = Field(default_factory=uuid.uuid4) 
    #id: uuid.UUID = Field(default_factory=lambda: uuid.uuid4()) # using lambda is also possible here, though a bit longer
    name: str

try:
    user = User(name="Allison")
    print(user)
    user = User(id = uuid.uuid4(), name="Bob") # we still can assign id explicitly
    print(user)
except ValidationError as e:
    print(e)

id=UUID('0caa0b7a-98a2-419d-a9cd-c7468aa0237a') name='Allison'
id=UUID('885a6bf7-44f5-46b8-b9d0-152155585a05') name='Bob'


Immutable attributes

In [22]:
from pydantic import BaseModel, ValidationError

class User(BaseModel):
    name: str
    age: int

    class Config:
        frozen = True

try:
    user = User(name="Alice", age=22)
    user.name = "Billy"
    print(user.name)
except ValidationError as e:
    print(e)

1 validation error for User
name
  Instance is frozen [type=frozen_instance, input_value='Billy', input_type=str]
    For further information visit https://errors.pydantic.dev/2.5/v/frozen_instance


In [23]:
from pydantic import BaseModel, ValidationError, ConfigDict

class User(BaseModel):
    model_config: ConfigDict = {"frozen": True}
    name: str
    age: int

try:
    user = User(name="Alice", age=22)
    user.name = "Billy"
    print(user.name)
except ValidationError as e:
    print(e)

1 validation error for User
name
  Instance is frozen [type=frozen_instance, input_value='Billy', input_type=str]
    For further information visit https://errors.pydantic.dev/2.5/v/frozen_instance


In [28]:
from pydantic import BaseModel, ValidationError
import uuid

class User(BaseModel):
    id: int = Field(default_factory=uuid.uuid4, frozen=True)
    name: str
    age: int

try:
    user = User(name="Alice", age=22)
    user.id = uuid.uuid4()
    print(user)
except ValidationError as e:
    print(e)

1 validation error for User
id
  Field is frozen [type=frozen_field, input_value=UUID('e847fe9d-06b4-4a58-8e42-be885dff2a1e'), input_type=UUID]
    For further information visit https://errors.pydantic.dev/2.5/v/frozen_field


Additional properties

In [32]:
from pydantic import BaseModel, Field
from uuid import UUID, uuid4

class Product(BaseModel):
    id: UUID = Field(frozen=True, default_factory=uuid4)
    name: str

try:
    product = Product(name="Chair", price=202.4) # pydantic just ignores irrelevant property
    print(product)
except ValueError as e:
    print(e)

id=UUID('37ea816c-94fc-44aa-9613-d110ad68acff') name='Chair'


In [37]:
from pydantic import BaseModel, Field
from uuid import UUID, uuid4

class Product(BaseModel):
    # model_config: ConfigDict = {"extra": "ignore"} # default value
    model_config: ConfigDict = {"extra": "forbid"}
    id: UUID = Field(frozen=True, default_factory=uuid4)
    name: str

try:
    # pydantic raises an exception because extra attributes are explicitly forbidden by config
    product = Product(name="Chair", price=202.4) 
    print(product)
except ValidationError as e:
    print(e)

1 validation error for Product
price
  Extra inputs are not permitted [type=extra_forbidden, input_value=202.4, input_type=float]
    For further information visit https://errors.pydantic.dev/2.5/v/extra_forbidden


In [39]:
from pydantic import BaseModel, Field
from uuid import UUID, uuid4

class Product(BaseModel):
    # model_config: ConfigDict = {"extra": "ignore"} # default value
    model_config: ConfigDict = {"extra": "allow"}
    id: UUID = Field(frozen=True, default_factory=uuid4)
    name: str

try:
    # pydantic raises no exception because extra attributes are explicitly allowed by config
    product = Product(name="Chair", price=202.4, weight=33) 
    # additional attributes are printed
    print(product)
except ValidationError as e:
    print(e)

id=UUID('e44093cc-28ef-4ba3-ab9d-610d6af8a856') name='Chair' price=202.4 weight=33
